This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [9]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

vKTNADnfZyEVhNeyoa-R


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to use Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [10]:
# First, import the relevant modules

In [11]:
import requests
import json
import os
from datetime import datetime
from dotenv import load_dotenv

Note: API's can change a bit with each version, for this exercise it is recommended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [12]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

In [13]:
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json"
params = {
    'api_key': API_KEY
}

In [14]:
response = requests.get(url, params=params)

In [15]:
if response.status_code == 200:
    data = response.json()
else:
    print("Failed to retrieve data", response.status_code)

In [16]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [17]:
columns = data['dataset']['column_names']
dataset = data['dataset']['data']
filtered_data = []

In [18]:
start_date = datetime.strptime('2017-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2017-12-31', '%Y-%m-%d')

In [19]:
# Collect 2017 data and save to dict

In [20]:
json_list = []

In [21]:
for record in dataset:
    record_date = datetime.strptime(record[0], '%Y-%m-%d')
    if start_date <= record_date <= end_date:
        json_dict = dict(zip(columns, record))
        json_list.append(json_dict)

In [22]:
# Calculate highest/lowest opening prices

In [23]:
lowest_open = float('inf')
highest_open = float('-inf')

In [24]:
for json_dict in json_list:
    open_price = json_dict.get('Open')
    if open_price is not None:
        open_price = float(open_price)
        if open_price < lowest_open:
            lowest_open = open_price
        if open_price > highest_open:
            highest_open = open_price

In [25]:
print("Highest opening price in 2017: {:.2f}".format(highest_open))
print("Lowest opening price in 2017: {:.2f}".format(lowest_open))

Highest opening price in 2017: 53.11
Lowest opening price in 2017: 34.00


In [26]:
# Largest change in any one day

In [27]:
largest_change = float('-inf')

In [28]:
for json_dict in json_list:
    high_price = float(json_dict.get("High"))
    low_price = float(json_dict.get("Low"))
    daily_change = abs(high_price - low_price)

    if daily_change > largest_change:
        largest_change = daily_change

print("Largest change in any one day: {:.2f}".format(largest_change))

Largest change in any one day: 2.81


In [29]:
# Largest change between any two days

In [30]:
for json_dict in json_list:
    close_price = json_dict.get('Close')

In [31]:
largest_closing_change = 0
previous_closing_price = None

In [32]:
for json_dict in json_list:
    closing_price = json_dict.get("Close")

    if closing_price is not None:
        closing_price = float(closing_price)
        if previous_closing_price is not None:
            daily_change = abs(closing_price - previous_closing_price) 
            if daily_change > largest_closing_change:
                largest_closing_change = daily_change
        
        previous_closing_price = closing_price

In [33]:
print("Largest change between consecutive days: {:.2f}".format(largest_closing_change))

Largest change between consecutive days: 2.56


In [34]:
# Avg daily trading volume

In [35]:
total_trading_volume = 0
trading_days_count = 0

In [36]:
trading_volumes = []

In [37]:
for json_dict in json_list:
    trading_volume = json_dict.get("Traded Volume")

    trading_volume = float(trading_volume)
    trading_volumes.append(trading_volume) # save for next step
    total_trading_volume += trading_volume
    trading_days_count += 1

In [38]:
average_trading_volume = total_trading_volume / trading_days_count
print("Average daily trading volume in 2017: {:.2f}".format(average_trading_volume))

Average daily trading volume in 2017: 89124.34


In [39]:
# Median trading volume this year

In [40]:
trading_volumes.sort()

In [41]:
print(trading_volumes)

[45.0, 112.0, 290.0, 522.0, 587.0, 686.0, 1081.0, 1945.0, 27507.0, 34640.0, 37014.0, 37958.0, 38308.0, 38518.0, 38834.0, 39195.0, 39731.0, 40660.0, 40705.0, 40800.0, 41911.0, 42669.0, 43042.0, 43362.0, 43375.0, 43976.0, 43988.0, 44037.0, 44059.0, 44700.0, 44803.0, 45111.0, 45263.0, 46070.0, 46510.0, 47130.0, 47730.0, 47879.0, 47999.0, 48123.0, 48152.0, 48412.0, 48706.0, 48797.0, 49180.0, 49246.0, 49863.0, 49970.0, 50350.0, 50549.0, 50557.0, 50587.0, 51321.0, 52828.0, 54107.0, 54279.0, 54408.0, 55333.0, 55707.0, 55799.0, 56071.0, 56224.0, 56240.0, 56245.0, 56319.0, 56319.0, 56906.0, 56911.0, 57452.0, 57838.0, 59367.0, 59881.0, 59991.0, 60324.0, 60420.0, 60583.0, 60716.0, 61925.0, 62225.0, 62297.0, 62655.0, 62672.0, 63371.0, 64103.0, 64320.0, 64334.0, 65157.0, 65213.0, 65266.0, 65546.0, 65837.0, 65847.0, 65860.0, 65931.0, 66116.0, 66351.0, 66760.0, 67342.0, 67375.0, 67410.0, 67440.0, 67593.0, 67672.0, 67781.0, 68801.0, 68812.0, 69455.0, 69603.0, 69657.0, 69734.0, 70056.0, 70269.0, 70618.

In [42]:
n = len(trading_volumes)

In [43]:
if n % 2 == 1:
    median_trading_volume = trading_volumes[n // 2]
else:
    median_trading_volume = (trading_volumes[n // 2 - 1] + trading_volumes[n // 2]) / 2

In [44]:
print("Median trading volume in 2017 {:.2f}".format(median_trading_volume))

Median trading volume in 2017 76286.00
